In [1]:
%load_ext autoreload
%autoreload 2

import torch
import torchvision
import torch.nn.functional as F
from torch import nn
from sklearn.metrics import precision_recall_fscore_support
import numpy as np

# manage ray's relative imports
# import ray
# runtime_env = {"working_dir": ".." }
# ray.init(runtime_env=runtime_env, dashboard_port=13065, include_dashboard=True)

from ray import tune
from ray.tune.suggest.optuna import OptunaSearch
from ray.tune import JupyterNotebookReporter

# manage beams's relative imports
import sys
sys.path.append('..')

from src.beam import beam_arguments, Experiment, Study
from src.beam import UniversalDataset, UniversalBatchSampler, PackedFolds
from src.beam import Algorithm
from src.beam import LinearNet, check_type, slice_to_index
from torchvision import transforms
import matplotlib.pyplot as plt

from src.beam import DataTensor
from src.beam.utils import is_notebook

from sklearn.datasets import fetch_covtype
import pandas as pd

from covtype_example import CovtypeDataset, CovtypeAlgorithm, RuleNet, covtype_algorithm_generator

In [7]:
path_to_data = '/home/shared/data/dataset/covtype'
root_dir = '/home/shared/data/results/covtype'

In [8]:
args = beam_arguments(
        f"--project-name=covtype --root-dir={root_dir} --algorithm=CovtypeAlgorithm --device=1 --no-half --lr-d=1e-3 --batch-size=256",
        "--n-epochs=100 --clip=0 --parallel=1 --accumulate=1 --cudnn-benchmark",
        "--weight-decay=1e-5 --beta1=0.9 --beta2=0.999", label_smoothing=.05, weight_factor=.5,
        path_to_data=path_to_data, gamma=1., dropout=.0, activation='gelu', channels=512)

experiment = Experiment(args)

2022-06-22 16:00:22.630 | INFO     | src.beam.experiment:__init__:183 - beam project: covtype
2022-06-22 16:00:22.633 | INFO     | src.beam.experiment:__init__:184 - Experiment Hyperparameters
2022-06-22 16:00:22.633 | INFO     | src.beam.experiment:__init__:189 - project_name: covtype
2022-06-22 16:00:22.634 | INFO     | src.beam.experiment:__init__:189 - identifier: debug
2022-06-22 16:00:22.634 | INFO     | src.beam.experiment:__init__:189 - algorithm: CovtypeAlgorithm
2022-06-22 16:00:22.635 | INFO     | src.beam.experiment:__init__:189 - mp_port: None
2022-06-22 16:00:22.638 | INFO     | src.beam.experiment:__init__:189 - root_dir: /home/shared/data/results/covtype
2022-06-22 16:00:22.638 | INFO     | src.beam.experiment:__init__:189 - reload: False
2022-06-22 16:00:22.638 | INFO     | src.beam.experiment:__init__:189 - resume: -1
2022-06-22 16:00:22.639 | INFO     | src.beam.experiment:__init__:189 - override: False
2022-06-22 16:00:22.639 | INFO     | src.beam.experiment:__init_

In [ ]:
alg = experiment(covtype_algorithm_generator)

2022-06-22 16:00:24.643 | INFO     | src.beam.experiment:run:553 - Single worker mode
2022-06-22 16:00:24.646 | INFO     | src.beam.experiment:run_worker:45 - Worker: 1/1 is running...


train:   0%|          | 0/1360 [00:00<?, ?it/s]

validation:   0%|          | 0/452 [00:00<?, ?it/s]


2022-06-22 16:06:09.499 | INFO     | src.beam.experiment:save_model_results:397 - Finished epoch 1/100:
2022-06-22 16:06:09.505 | INFO     | src.beam.experiment:log_data:443 - train:
2022-06-22 16:06:09.506 | INFO     | src.beam.experiment:log_data:447 - loss 221.69 	|
2022-06-22 16:06:09.507 | INFO     | src.beam.experiment:log_data:447 - acc 0.705762 	|
2022-06-22 16:06:09.507 | INFO     | src.beam.experiment:log_data:447 - lr 0.001 	|
2022-06-22 16:06:09.508 | INFO     | src.beam.experiment:log_data:443 - validation:
2022-06-22 16:06:09.509 | INFO     | src.beam.experiment:log_data:447 - loss 195.039 	|
2022-06-22 16:06:09.509 | INFO     | src.beam.experiment:log_data:447 - acc 0.761235 	|


train:   0%|          | 0/1360 [00:00<?, ?it/s]

validation:   0%|          | 0/452 [00:00<?, ?it/s]


2022-06-22 16:11:53.927 | INFO     | src.beam.experiment:save_model_results:397 - Finished epoch 2/100:
2022-06-22 16:11:53.935 | INFO     | src.beam.experiment:log_data:443 - train:
2022-06-22 16:11:53.936 | INFO     | src.beam.experiment:log_data:447 - loss 163.406 	|
2022-06-22 16:11:53.936 | INFO     | src.beam.experiment:log_data:447 - acc 0.824204 	|
2022-06-22 16:11:53.937 | INFO     | src.beam.experiment:log_data:447 - lr 0.001 	|
2022-06-22 16:11:53.937 | INFO     | src.beam.experiment:log_data:443 - validation:
2022-06-22 16:11:53.938 | INFO     | src.beam.experiment:log_data:447 - loss 153.522 	|
2022-06-22 16:11:53.938 | INFO     | src.beam.experiment:log_data:447 - acc 0.843154 	|


train:   0%|          | 0/1360 [00:00<?, ?it/s]